In [1]:
import torch
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

%load_ext autoreload
%autoreload 2

from deep_cnn_structures import ResNet

In [2]:
batch_size = 16
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, 
                                       transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                          num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,
                                      transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False,
                                         num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
net = ResNet()
net.cuda()

ResNet(
  (conv1A): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn1A): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1))
  (layer2A): Bottleneck1(
    (conv1): Conv2d (64, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv3): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1))
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (conv_sc): Conv2d (64, 256, kernel_size=(1, 1), stride=(1, 1))
    (bn_sc): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  )
  (layer2B): Bottleneck2(
    (conv1): Conv2d (256, 64, kernel_size=(1, 1), stride=(1, 1))
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (conv2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

In [6]:
for epoch in range(20):
    running_loss = 0.
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        
        if (i+1) % 500 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/500))
            running_loss = 0.

print('Finished Training')

[1,   500] loss: 0.252
[1,  1000] loss: 0.280
[1,  1500] loss: 0.303
[2,   500] loss: 0.185
[2,  1000] loss: 0.212
[2,  1500] loss: 0.231
[3,   500] loss: 0.129
[3,  1000] loss: 0.171
[3,  1500] loss: 0.187
[4,   500] loss: 0.092
[4,  1000] loss: 0.121
[4,  1500] loss: 0.145
[5,   500] loss: 0.083
[5,  1000] loss: 0.111
[5,  1500] loss: 0.131
[6,   500] loss: 0.068
[6,  1000] loss: 0.093
[6,  1500] loss: 0.107
[7,   500] loss: 0.064
[7,  1000] loss: 0.075
[7,  1500] loss: 0.089
[8,   500] loss: 0.052
[8,  1000] loss: 0.075
[8,  1500] loss: 0.074
[9,   500] loss: 0.053
[9,  1000] loss: 0.062
[9,  1500] loss: 0.071
[10,   500] loss: 0.040
[10,  1000] loss: 0.062
[10,  1500] loss: 0.070
[11,   500] loss: 0.048
[11,  1000] loss: 0.059
[11,  1500] loss: 0.057
[12,   500] loss: 0.034
[12,  1000] loss: 0.062
[12,  1500] loss: 0.055
[13,   500] loss: 0.035
[13,  1000] loss: 0.049
[13,  1500] loss: 0.057
[14,   500] loss: 0.036
[14,  1000] loss: 0.053
[14,  1500] loss: 0.048
[15,   500] loss: 0

In [7]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()
    
print('Accuracy of the network on 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on 10000 test images: 82 %


In [15]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for data in testloader:
    images, labels = data
    outputs = net(Variable(images.cuda()))
    _, predicted = torch.max(outputs.data, 1)
    c = (predicted == labels.cuda())
    for i in range(labels.size(0)):
        label = labels[i]
        class_correct[label] += c[i]
        class_total[label] += 1
        
for i in range(10):
    print('Accuracy of %5s : %2d %%' % (classes[i], 100*class_correct[i]/class_total[i]))

Accuracy of plane : 86 %
Accuracy of   car : 91 %
Accuracy of  bird : 69 %
Accuracy of   cat : 71 %
Accuracy of  deer : 85 %
Accuracy of   dog : 69 %
Accuracy of  frog : 89 %
Accuracy of horse : 84 %
Accuracy of  ship : 90 %
Accuracy of truck : 86 %


In [ ]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    
dataiter = iter(trainloader)
images, labels = dataiter.next()

dataiter = iter(testloader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

In [ ]:
outputs = net(Variable(images.cuda()))
_, predicted = torch.max(outputs.data, 1)
print('Predicted: ', ' '.join('%5s' % classes[predicted[j]] for j in range(batch_size)))